# WNTR Landslide Tutorial
The following tutorial illustrates the use of the `wntr.gis` module to use landslide geospatial data in a resilience analysis. The objective of this tutorial is to 1) quantify water service disruptions that could occur from pipes damaged in landslides and 2) identify population impacted by the service disruptions.

To simplify the tutorials, it is assumed that pipes within a 1000 ft buffer of each landslide susceptible region are damaged in that landslide.
This assumption could be replaced with detailed landslide analysis that includes slope, soil type, weather conditions, and pipe material.

## Imports
Import WNTR and additional Python packages that are needed for the tutorial.
- Geopandas is used to load geospatial data
- Shapely is used to define a region of interest to crop data
- Matplotlib is used to create graphics

In [ ]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pylab as plt
import wntr

## Units
WNTR uses SI (International System) units (length in meters, time in seconds, mass in kilograms), **with the exception of the landslide buffer which is in feet to match the coordinate reference system of the geospatial data**.  See https://usepa.github.io/WNTR/units.html for more details on WNTR units.

In [ ]:
# Define the coordinates used to zoom in on network graphics
zoom_coords = [(5.75e6, 5.79e6), (3.82e6, 3.85e6)] 

# Water network model
The following section creates a `WaterNetworkModel` object from an EPANET input (INP) file and converts the model to GeoDataFrames for use in geospatial analysis.

## Create a water network model
The drinking water distribution network model used in this tutorial was downloaded from the [UKnowledge Water Distribution Systems Research Database](https://uknowledge.uky.edu/wdsrd/). KY 10 was selected for the analysis. The following section creates a `WaterNetworkModel` from an EPANET INP file and calculates some general attributes of the model.

*Citation: Hoagland, Steven, "10 KY 10" (2016). Kentucky Dataset. 12. https://uknowledge.uky.edu/wdst/12. Accessed on 4/4/2024.*

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
inp_file = 'networks/ky10.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

In [ ]:
# Print a basic description of the model 
# The level can be 0, 1, or 2 and defines the level of detail included in the description
wn.describe(level=1)

In [ ]:
# Calculate the total pipe length in meters 
length = wn.query_link_attribute('length')
total_length = length.sum() # m
# Print the total pipe length in meters and feet
print('Total pipe length =', round(total_length,2), 'm, =', round(total_length*3.28084,2), 'ft')

In [ ]:
# Calculate the average expected demand in cubic meters
average_expected_demand = wntr.metrics.average_expected_demand(wn) # m^3/s
# Calculate the average volume per day in cubic meters
average_volume_per_day = average_expected_demand*(24*3600) # m^3
# Calculate the total water use in cubic meters
total_water_use = average_volume_per_day.sum() # m^3
# Print the total water use in cubic meters and million gallons
print('Total water use =', round(total_water_use,2), 'm^3 =', round(total_water_use*264.172/1e6,2), 'million gallons')

In [ ]:
# Calculate the population served at each junction using the default average volume of water consumed per capita per day of 200 gallons/day
population = wntr.metrics.population(wn) 
# Calcualte the total population served by the network
total_population = population.sum()
# Print the total population served by the network
print('Total population =', total_population)

In [ ]:
# Plot the elevation for each junction on the network 
# Note that the remaining graphics in this tutorial are created from the geospatial data directly, rather than the `plot_network` function
# The `plot_network` function currently does not include vertices
ax = wntr.graphics.plot_network(wn, node_attribute='elevation', node_range=(175, 300), title='KY 10 elevation', node_colorbar_label='Elevation (m)')

## Convert the WaterNetworkModel to geographic information system (GIS) data
The `WaterNetworkModel` is converted to a collection of GIS compatible GeoDataFrames and the coordinate reference system (CRS) is set to **EPSG:3089 NAD83 / Kentucky Single Zone (ftUS)** (see https://epsg.io/3089 for more details). Data for junctions, tanks, reservoirs, pipes, pumps, and valves are stored in separate GeoDataFrames.

In [ ]:
# Convert the WaterNetworkModel to GIS data
wn_gis = wn.to_gis()
# Print only the first 5 junction entries
# Example commands are provided for all of the network compoment conversions
wn_gis.junctions.head()
#wn_gis.tanks.head()
#wn_gis.reservoirs.head()
#wn_gis.pipes.head()
#wn_gis.pumps.head()
#wn_gis.tanks.head()


In [ ]:
# Set the CRS to EPSG:3089 NAD83 / Kentucky Single Zone (ftUS)
crs = 'EPSG:3089'
wn_gis.set_crs(crs)

In [ ]:
# Plot the network using the GIS data 
fig, ax = plt.subplots(figsize=(5,5))
ax = wn_gis.pipes.plot(column='diameter', linewidth=1, label='pipes', alpha=0.8, ax=ax, zorder=1)
ax = wn_gis.reservoirs.plot(color='k', marker='s', markersize=60, label='reservoirs', ax=ax)
ax = wn_gis.tanks.plot(color='r', markersize=20, label='tanks', ax=ax)
ax = wn_gis.pumps.centroid.plot(color='b', markersize=20, label='pumps', ax=ax)
ax = wn_gis.valves.centroid.plot(color='c', markersize=20, label='valves', ax=ax)
tmp = ax.set_title('KY 10')
# Comment/uncomment the following 3 lines to change border/axis around the graphic
#tmp = ax.axis('off')
tmp = ax.set_xticks([])
tmp = ax.set_yticks([])
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
#tmp = ax.set_xlim(zoom_coords[0])
#tmp = ax.set_ylim(zoom_coords[1])
tmp = plt.legend()

## Save the GIS data to GeoJSON or Shapefile files
The GIS data can be written to GeoJSON files or Shapefile files. One file is created for each of the network components: junctions, tanks, reservoirs, pipes, pumps, and valves (e.g., ky10_junctions.geojson, ky10_tanks.geojson). The GeoJSON or Shapefile files can be loaded into GIS software platforms for further analysis. **Note that controls, patterns, curves, and options are not included in the GIS formatted data files.** 

In [ ]:
# Store the WaterNetworkModel as a collection of GeoJSON files
# Check project folder to verify that files were written 
wn_gis.write_geojson('ky10')

# Landslide GIS data
Landslide inventory data is loaded and the impacted areas are expanded to illustrate landslide scenarios. 

## Load landslide GIS data
The landslide data used in this tutorial was downloaded from the [UKnowledge Kentucky Geological Survey Research Data](https://uknowledge.uky.edu/kgs_data/).  The Kentucky Geological Survey Landslide Inventory from March 2023 was selected for the analysis.  The data contains locations of known landslides and areas susceptible to debris flows, derived from aerial photography. 

*Citation: Crawford, M.M., 2023. Kentucky Geological Survey landslide inventory [2023-03]: Kentucky Geological Survey Research Data, https://uknowledge.uky.edu/kgs_data/7/, Accessed on 4/4/2024.*

In [ ]:
# To reduce the file size checked into the WNTR repository, the following code was run on the raw data file
# This example can be copied and pasted into a .py file in your project and run on any geodatabase file to truncate the data to the region of interest (ROI)
# The ROI includes a 5000 ft buffer surrounding all pipes. The function `box` was imported from shapely
"""
bounds = wn_gis.pipes.total_bounds # total_bounds returns the upper and lower bounds on x and y
geom = box(*bounds)
ROI = geom.buffer(5000) # feet

landslide_file = 'data/KGS_Landslide_Inventory_exp.gdb'
landslide_data = gpd.read_file(landslide_file, driver="FileGDB", layer='Areas_derived_from_aerial_photography')
print(landslide_data.crs)
landslide_data = landslide_data.clip(ROI)
landslide_data.to_file("data/ky10_landslide_data.geojson", index=True, driver='GeoJSON')
"""

In [ ]:
# Load the landslide data from file and print the CRS to ensure it is in EPSG:3089  
# The methods `to_crs` and `set_crs` can be used to change coordinate reference systems if needed
landslide_file = 'data/ky10_landslide_data.geojson'
landslide_data = gpd.read_file(landslide_file).set_index('index') 
# Print the CRS for the landslide data
print(landslide_data.crs)
# Print the first 5 entries of landslide data
landslide_data.head()

In [ ]:
# Plot the landslide data along with pipes
ax = landslide_data.plot(color='red', label='Landslide data')
ax = wn_gis.pipes.plot(color='black', linewidth=1, ax=ax, label='Pipes')
ax.set_title('Landslide and pipe data')
# Comment/uncomment the following 3 lines to change border/axis around the graphic
#tmp = ax.axis('off')
tmp = ax.set_xticks([])
tmp = ax.set_yticks([])
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])

## Expand the size of each landslide using a buffer
Each landslide is extended to include the surrounding 1000 ft to create a region that might be impacted by an individual landslide. The distance unit for buffering matches the distance unit of the CRS (ft).
This assumption could be replaced with detailed landslide analysis that includes slope, soil type, weather conditions, and pipe material. 

In [ ]:
# Create a GeoDataFrame to hold information used in landslide scenarios (initially copied from landslide_data)
# Buffer each landslide polygon by 1000 ft
landslide_scenarios = landslide_data.copy()
landslide_scenarios['geometry'] = landslide_data.buffer(1000)

In [ ]:
# Add a prefix to the landslide scenario index to indicate the scenario name
landslide_scenarios.index = 'LS-' + landslide_scenarios.index.astype(str)

In [ ]:
# Plot the landslide data, the buffers of each landslide scenario, and the pipes
ax = landslide_scenarios.plot(color='gray', alpha=0.5, label='Landslide buffer')
ax = landslide_data.plot(color='red', label='Landslide data', ax=ax)
ax = wn_gis.pipes.plot(color='black', linewidth=1, ax=ax, label='Pipes')
ax.set_title('Landslide scenarios and pipe data')
# Comment/uncomment the following 3 lines to change border/axis around the graphic
#tmp = ax.axis('off')
tmp = ax.set_xticks([])
tmp = ax.set_yticks([])
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])

# Geospatial intersects
In this section, landslide scenario data are intersected with pipes and junctions in the `WaterNetworkModel`.

## Identify pipes that intersect each landslide
Landslide polygons are intersected with pipes to obtain a list of pipes that intersect each landslide.  <font color='red'>This information is used to to define the pipes that are closed in each landslide scenario.</font> The pipe attribute `length` is also included in the intersection to gather statistics on the pipe length that intersects each landslide.  

In [ ]:
# Use the intersect function to determine pipes and pipe length that intersects each landslide scenario
A = landslide_scenarios
B = wn_gis.pipes
B_value = 'length'
landslide_intersect = wntr.gis.intersect(A, B, B_value)

# Print results in order of descending total pipe length for the first 5 entries only 
landslide_intersect.sort_values('sum', ascending=False).head()

In [ ]:
# Add the intersection results to the landslide scenario data
landslide_scenarios[['intersections', 'n', 'total pipe length']] = landslide_intersect[['intersections', 'n', 'sum']]

# Print results in order of descending total pipe length for the first 5 entries only 
landslide_scenarios.sort_values('total pipe length', ascending=False).head()

In [ ]:
# Plot intersection results
fig, axes = plt.subplots(1,2, figsize=(15,5))

wn_gis.pipes.plot(color='gray', linewidth=1, ax=axes[0])
landslide_scenarios.plot(column='n', vmax=10, legend=True, ax=axes[0], legend_kwds={'label':'Number of pipes'})
tmp = axes[0].set_title('Number of pipes that intersect each landslide')
# Comment/uncomment the following 3 lines to change border/axis around the graphic
#tmp = axes[0].axis('off')
tmp = axes[0].set_xticks([])
tmp = axes[0].set_yticks([])
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = axes[0].set_xlim(zoom_coords[0])
tmp = axes[0].set_ylim(zoom_coords[1])

wn_gis.pipes.plot(color='gray', linewidth=1, ax=axes[1])
landslide_scenarios.plot(column='total pipe length', vmax=10000, legend=True, ax=axes[1], legend_kwds={'label':'Pipe length (m)'})
tmp = axes[1].set_title('Length of pipe that intersect each landslide')
# Comment/uncomment the following 3 lines to change border/axis around the graphic
#tmp = axes[1].axis('off')
tmp = axes[1].set_xticks([])
tmp = axes[1].set_yticks([])
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = axes[1].set_xlim(zoom_coords[0])
tmp = axes[1].set_ylim(zoom_coords[1])

## Identify landslides that intersect each pipe
The WNTR intersect function is used to obtain a list of landslides that intersect each pipe to identify landslide impacts on the network. The landslide attribute `Confidence_Ranking` is also included in the intersection to gather statistics on landslide confidence for each pipe.  <font color='red'>While this information is not used in the analysis below, this type of information could be used to inform uncertainty or probability of damage.</font>

**Note that `Confidence_Ranking` has a value of 3 ("Landslide likely at or near the specified location") for each landslide in the region of interest. Since the values are uniform in this dataset, the intersected sum, min, max, and mean are all the same value.** More information on confidence ranking can be found at https://kgs.uky.edu/kgsmap/helpfiles/landslide_help.shtm.  

In [ ]:
# Use the intersect function to determine landslides and landslide confidence ranking that intersects each pipe
A = wn_gis.pipes
B = landslide_scenarios
B_value = 'Confidence_Ranking'
pipe_intersect = wntr.gis.intersect(A, B, B_value)

# Print results in order of descending number of intersections for the first 5 entries only 
pipe_intersect.sort_values('n', ascending=False).head()

In [ ]:
# Add the intersection results to the GIS pipe data
wn_gis.pipes[['intersections', 'n', 'Confidence_Ranking']] = pipe_intersect[['intersections', 'n', 'mean']]

# Print results in order of descending number of intersections for the first 5 entries only 
wn_gis.pipes.sort_values('n', ascending=False).head()

In [ ]:
# Plot intersection results
ax = wn_gis.pipes.plot(color='gray', linewidth=1, zorder=1)
wn_gis.pipes[wn_gis.pipes['n'] > 0].plot(column='n', vmax=20, legend=True, ax=ax, legend_kwds={'label':'Number of landslides'})
tmp = ax.set_title('Number of landslide scenarios that intersect each pipe', fontdict={'fontsize':10})
# Comment/uncomment the following 3 lines to change border/axis around the graphic
#tmp = ax.axis('off')
tmp = ax.set_xticks([])
tmp = ax.set_yticks([])
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])

# Hydraulic simulations
The following section simulates the hydraulics for the baseline (no landslide) and landslide scenarios. A subset of landslide scenarios is run to simplify the tutorial. Simulation results are stored for later analysis.

In [ ]:
# Create a function to setup the WaterNetworkModel for hydraulic simulations
def model_setup(inp_file):
    wn = wntr.network.WaterNetworkModel(inp_file)
    wn.options.hydraulic.demand_model = 'PDD'
    wn.options.hydraulic.required_pressure = 20 # m
    wn.options.hydraulic.minimum_pressure  = 0 # m
    wn.options.time.duration = 48*3600 # s (48 hour simulation)
    return wn

## Baseline scenario

In [ ]:
# Simulate the baseline hydraulics with no landslide or damage using EPANET 
wn = model_setup(inp_file)
sim = wntr.sim.EpanetSimulator(wn)
baseline_results = sim.run_sim()

In [ ]:
# Print the pressures for each junction for only the first 5 timesteps 
baseline_results.node['pressure'].head()

## Landslide scenarios
Landslide scenarios are downselected by identifying the set of landslides that impact a unique set of pipes. Scenarios are further downselected to six scenarios to simplify the tutorial. A hydraulic simulation is run for each landslide scenario, where pipes that intersect the landslide are closed for 48 hours. Results from each scenario are stored for later analysis.

In [ ]:
# Remove scenarios with no intersecting pipes
landslide_scenarios = landslide_scenarios[landslide_scenarios['n'] > 0]
landslide_scenarios = landslide_scenarios[~landslide_scenarios['n'].isna()]

# Select landslide scenarios that impact a unique set of pipes (i.e., remove duplicative pipe entries)
duplicated_intersections = landslide_scenarios['intersections'].astype(str).duplicated()
landslide_scenarios = landslide_scenarios.loc[~duplicated_intersections, :]

# Print the number of unique landslide scenarios for the first 5 entries only 
print('Number of unique landslide scenarios', landslide_scenarios.shape[0])
landslide_scenarios.head()

In [ ]:
# Further downselect the landslide scenarios for demonstration purposes
# Uncomment one option at a time to explore that option

# Option 1. Six scenarios that illustrate a wide range of impact
landslide_scenarios_downselect = landslide_scenarios.loc[['LS-4495', 'LS-7003', 'LS-7111', 'LS-5086', 'LS-6966', 'LS-7058'],:] 

# Option 2. Six scenarios with the highest intersecting pipe length
#landslide_scenarios_downselect = landslide_scenarios.sort_values('total pipe length', ascending=False).iloc[0:6,:]

# Option 3. Six scenarios with the highest number of intersecting pipes
#landslide_scenarios_downselect = landslide_scenarios.sort_values('n', ascending=False).iloc[0:6,:]

# Option 4. Random selection of six scenarios
#landslide_scenarios_downselect = landslide_scenarios.sample(n=6, random_state=1234)

# Print the landslide scenarios selected
landslide_scenarios_downselect

In [ ]:
# Plot the location of landslides used in the analysis
ax = landslide_scenarios_downselect.plot(color='blue')
wn_gis.pipes.plot(color='gray', linewidth=1, ax=ax)
tmp = ax.set_title('Landslide scenarios')
# Comment/uncomment the following 3 lines to change border/axis around the graphic
#tmp = ax.axis('off')
tmp = ax.set_xticks([])
tmp = ax.set_yticks([])

In [ ]:
# Simulate the hydraulics for each landslide scenario using EPANET
# Store simulation results in a dictionary
# Each scenario closes all pipes that intersect the landslide for the 48 hour simulation
results = {}
for i, scenario in landslide_scenarios_downselect.iterrows():
    wn = model_setup(inp_file)
    for pipe_i in scenario['intersections']:
        pipe_object = wn.get_link(pipe_i)
        pipe_object.initial_status = 'CLOSED'
    sim = wntr.sim.EpanetSimulator(wn)
    results[i] = sim.run_sim()

# Analysis results
The following section calculates and plots analysis results, including water service availability (WSA) of impacted junctions for each scenario.

## Water service availability (WSA)
Water service availability (WSA) is the ratio of delivered demand to the expected demand. WSA is calculated for each junction (alternatively, WSA can be calculated for each timestep, or for each junction and timestep). A value equal to 1 indicates that all expected demand is met, while a value of 0 indicates that the none of expected demand is met. 

**Note that WSA can be > 1 and < 0 due to numerical differences in expected and actual demand. For certain types of analysis, WSA should be truncated to values between 0 and 1.**

In [ ]:
# Calculate expected demand for each junction and timestep
expected_demand = wntr.metrics.expected_demand(wn)

# Print the expected demand for each junction for only the first 5 timesteps
expected_demand.round(2).head()

In [ ]:
# Calculate total expected demand at each junction (axis 0 is the time index)
expected_demand_j = expected_demand.sum(axis=0)

# Print the total expected demand for each junction
expected_demand_j

In [ ]:
# Calculate the baseline WSA for each junction
demand_baseline = baseline_results.node['demand'].loc[:,wn.junction_name_list]
demand_baseline_j = demand_baseline.sum(axis=0) # total demand at each junction
wsa_baseline_j = wntr.metrics.water_service_availability(expected_demand_j, demand_baseline_j)

# Print WSA for first 5 junctions
wsa_baseline_j.head()

In [ ]:
# Add WSA from the baseline simulation to the junction GIS data
wn_gis.junctions['baseline'] = wsa_baseline_j

In [ ]:
# Plot WSA from the baseline simulation for each junction on the network 
ax = wn_gis.pipes.plot(color='black', linewidth=1)
ax = wn_gis.junctions.plot(column='baseline', cmap='cividis', vmin=0, vmax=1, legend=True, ax=ax, legend_kwds={'label':'WSA'})
tmp = ax.set_title('Baseline WSA')
# Comment/uncomment the following 3 lines to change border/axis around the graphic
#tmp = ax.axis('off')
tmp = ax.set_xticks([])
tmp = ax.set_yticks([])

In [ ]:
# Calculate WSA associated with each landslide scenarios
for i, scenario in landslide_scenarios_downselect.iterrows():
    demand = results[i].node['demand'].loc[:,wn.junction_name_list]
    demand_j = demand.sum(axis=0) # total demand at each junction
    wsa_j = wntr.metrics.water_service_availability(expected_demand_j, demand_j)
    
    # Add WSA to the junction GIS data
    wn_gis.junctions[i] = wsa_j
    # Print landslide scenarion ID, number of intersected pipes, and WSA
    print(i, len(scenario['intersections']), wsa_j.mean())

In [ ]:
# Extract WSA for each landslide scenario 
wsa_results = wn_gis.junctions[landslide_scenarios_downselect.index]

# Print WSA for only first 5 junctions for each landslide scenario
wsa_results.head()

In [ ]:
# Plot the distribution of WSA for each landslide scenario 
# Note that WSA can be > 1 and < 0 due to numerical differences in expected and actual demand 
ax = wsa_results.boxplot()
tmp = ax.set_ylim(-0.25, 1.25)
tmp = ax.set_ylabel('WSA')
tmp = ax.set_xlabel('Landslide scenario')
tmp = ax.set_title('Distribution of WSA for each scenario')

In [ ]:
# Plot WSA from each landslide scenario for each junction on the network
fig, axes = plt.subplots(2,3, figsize=(15,10))
axes = axes.flatten()

for i, scenario in enumerate(wsa_results.columns):
    wn_gis.pipes.plot(color='gray', linewidth=1, ax=axes[i]) # pipes
    wn_gis.junctions.plot(column=scenario, cmap='cividis', vmin=0, vmax=1, legend=True, ax=axes[i], legend_kwds={'label':'WSA'}) # junction wsa
    tmp = axes[i].set_title('WSA '+scenario)
    # Comment/uncomment the following 3 lines to change border/axis around the graphic
    #tmp = axes[i].axis('off')
    tmp = axes[i].set_xticks([])
    tmp = axes[i].set_yticks([])
    if i >= 6: # axes is defined to have 6 subplots
        break

## WSA of impacted junctions
In this analysis, impacted junctions are defined as junctions where WSA falls below 0.5 (50% of the expected water was received) at any time during the simulation. Other criteria could also be used to define impact.

In [ ]:
# Extract junctions that are impacted by WSA < 0.5 for each landslide scenario
impacted_junctions = {}
for scenario in wsa_results.columns:
    filter = wsa_results[scenario] < 0.5
    impacted_junctions[scenario] = wsa_results.index[filter]

# Print a list of impacted junctions for each landslide scenario
impacted_junctions

In [ ]:
# Plot WSA for only the impacted junctions on the network for each landslide scenario
fig, axes = plt.subplots(2,3, figsize=(15,10))
axes = axes.flatten()

for i, scenario in enumerate(wsa_results.columns):
    j = impacted_junctions[scenario]
    wn_gis.pipes.plot(color='gray', linewidth=1, alpha=0.5, ax=axes[i]) # pipes
    if len(j) > 0:
        wn_gis.junctions.loc[j,:].plot(column=scenario, cmap='cividis', vmin=0, vmax=1, legend=True, ax=axes[i], legend_kwds={'label':'WSA'}) # junction wsa
    tmp = axes[i].set_title('WSA '+scenario)
    # Comment/uncomment the following 3 lines to change border/axis around the graphic
    #tmp = axes[i].axis('off')
    tmp = axes[i].set_xticks([])
    tmp = axes[i].set_yticks([])
    if i >= 6: # axes is defined to have 6 subplots
        break

## Save analysis results to GIS files
The analysis above stored WSA results for each scenario to the `wn_gis` object, which can be saved to GIS formatted files and loaded into GIS software platforms for further analysis.  **Note that lists (such as the information stored in 'intersections') is not JSON serializable and must first be removed.**

In [ ]:
# Save the results as a GeoJSON file
del wn_gis.pipes['intersections']
wn_gis.write_geojson('ky10_analysis_results')